In [2]:
import pandas as pd
import fastparquet
import sqlite3

In [33]:
input_data = pd.read_parquet('../test_files/merged_cow_with_feed_daily_20230530_1557.parquet')
animal_data = input_data.dropna(subset=['sampleId'])

## Original version of assigning Diet_ID

In [34]:
# Setup a test file with animal information and a diet ID
def assign_diet_id(row):
    return f"{row['sampleId']}_{row['reportDate']}"


new_column = animal_data.apply(assign_diet_id, axis=1)
# animal_data = animal_data.drop(columns='Diet_ID')
animal_data.insert(0, 'Diet_ID', new_column)


In [35]:
animal_data['Diet_ID']

1658     DAIRY - LACTATING TMR_2021-01-25 00:00:00
1659     DAIRY - LACTATING TMR_2021-01-25 00:00:00
1660     DAIRY - LACTATING TMR_2021-01-25 00:00:00
1661     DAIRY - LACTATING TMR_2021-01-25 00:00:00
1662     DAIRY - LACTATING TMR_2021-01-25 00:00:00
                           ...                    
17594      DAIRY LACTATING TMR_2021-12-10 00:00:00
17595      DAIRY LACTATING TMR_2021-12-10 00:00:00
17596      DAIRY LACTATING TMR_2021-12-10 00:00:00
17597      DAIRY LACTATING TMR_2021-12-10 00:00:00
17598      DAIRY LACTATING TMR_2021-12-10 00:00:00
Name: Diet_ID, Length: 15941, dtype: object

## Vectorised way:
This is a vectorised version of above because it can work on the whole columns instead of looping through each row at a time.
It also converts the 'datetime' to a string in a format we want, e.g. in this case it's YYYY-MM-DD  (written as '%Y-%m-%d' in code ).
This is much faster, doesn't require a separate function to be defined and doesn't edit the data frame in-place using .insert() 

In [36]:
testdf = animal_data.assign(
    #new_column
    Diet_ID = lambda df: df['sampleId'] + '_' + df['reportDate'].dt.strftime('%Y-%m-%d')
)

In [37]:
testdf['Diet_ID']

1658     DAIRY - LACTATING TMR_2021-01-25
1659     DAIRY - LACTATING TMR_2021-01-25
1660     DAIRY - LACTATING TMR_2021-01-25
1661     DAIRY - LACTATING TMR_2021-01-25
1662     DAIRY - LACTATING TMR_2021-01-25
                       ...               
17594      DAIRY LACTATING TMR_2021-12-10
17595      DAIRY LACTATING TMR_2021-12-10
17596      DAIRY LACTATING TMR_2021-12-10
17597      DAIRY LACTATING TMR_2021-12-10
17598      DAIRY LACTATING TMR_2021-12-10
Name: Diet_ID, Length: 15941, dtype: object

In [38]:
# cleanup animal data
animal_data = animal_data.drop(columns=animal_data.columns[23:50], axis=1)

## DB functions:

In [6]:
# Use Diet_ID to get the corresponding NDF value


conn = sqlite3.connect('../diet_database.db')
cursor = conn.cursor()

diet_id = 'DAIRY - LACTATING TMR_2021-01-25 00:00:00'  # Replace 'your_diet_id' with the actual Diet_ID value you want to retrieve

query = f"SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = '{diet_id}'"
cursor.execute(query)

column_data = cursor.fetchone()
column_data = float(column_data[0])
print(column_data)  # Display the retrieved column data

conn.close()

30.28


This function also loops and sends the same query over and over, I have added a print line that will show that it is sending a query for iteration (row)

In [39]:
# Excecute the above code using a function that can be applied to each row of a dataframe
# This is a good example of how to pull individual values from the database using the Diet_ID to call the
# appropriate diet

def retrieve_ndf_value(row, conn):
    cursor = conn.cursor()

    diet_id = row['Diet_ID'] 

    query = f"SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = '{diet_id}'"
    print(query)
    cursor.execute(query)

    column_data = cursor.fetchone()
    ndf_value = float(column_data[0])  
    
    return ndf_value

conn = sqlite3.connect('../diet_database.db')

# Apply the function to each row of the dataframe and assign the result to a new column
animal_data['NDF_%_DM'] = animal_data.apply(retrieve_ndf_value, args=(conn,), axis=1)

conn.close()

SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = 'DAIRY - LACTATING TMR_2021-01-25 00:00:00'
SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = 'DAIRY - LACTATING TMR_2021-01-25 00:00:00'
SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = 'DAIRY - LACTATING TMR_2021-01-25 00:00:00'
SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = 'DAIRY - LACTATING TMR_2021-01-25 00:00:00'
SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = 'DAIRY - LACTATING TMR_2021-01-25 00:00:00'
SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = 'DAIRY - LACTATING TMR_2021-01-25 00:00:00'
SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = 'DAIRY - LACTATING TMR_2021-01-25 00:00:00'
SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = 'DAIRY - LACTATING TMR_2021-01-25 00:00:00'
SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = 

In [40]:
animal_data['NDF_%_DM']

1658     30.28
1659     30.28
1660     30.28
1661     30.28
1662     30.28
         ...  
17594    29.88
17595    29.88
17596    29.88
17597    29.88
17598    29.88
Name: NDF_%_DM, Length: 15941, dtype: float64

# Faster version:
This version will send 1 query, which gets just the 2 columns we need (or we could SELECT multiple columns if we want),
Then we can use pd.merge() which will add the columns we retrieved by matching the values in the column we define with 'on = '.

In [41]:
conn = sqlite3.connect('../diet_database.db')
query = "SELECT Diet_ID, `Neutral Detergent Fibre (%)` FROM current_diets"
# create data frame from query:
df_NDF = pd.read_sql_query(query, conn)
conn.close()

# merge data:
animal_data_2 = pd.merge(
    animal_data,
    df_NDF,
    on = 'Diet_ID'
)


In [43]:
animal_data_2['Neutral Detergent Fibre (%)']

0        30.28
1        30.28
2        30.28
3        30.28
4        30.28
         ...  
15936    29.88
15937    29.88
15938    29.88
15939    29.88
15940    29.88
Name: Neutral Detergent Fibre (%), Length: 15941, dtype: float64

In [21]:
# view data frame that was retrieved with query
df_NDF

,Diet_ID,Neutral Detergent Fibre (%)
0,DAIRY - LACTATING TMR_2021-01-25 00:00:00,30.28
1,DAIRY-LACTATING TMR_2021-02-04 00:00:00,33.02
2,DARIRY LACTATING TMR_2021-03-05 00:00:00,33.45
3,DAIRY NON RUMENSIN LACTATING TMR_2021-04-08 00...,31.79
4,DAIRY LACTATING TMR_2021-05-06 00:00:00,31.82
5,NON RUM LACT TMR-DAIRY_2021-06-04 00:00:00,29.61
6,LACTATING TMR-DAIRY_2021-07-02 00:00:00,29.77
7,DAIRY LACTATING TMR_2021-08-06 00:00:00,28.25
8,DAIRY LACTATING TMR_2021-09-03 00:00:00,31.14
9,DAIRY LACTATING TMR_2021-10-12 00:00:00,30.97


In [45]:
# then to do a calculation for DMI:
animal_data_2 = animal_data_2.assign(
    NDF_kg_DM = lambda df: df['Neutral Detergent Fibre (%)']/100 * df['DMI']
)

animal_data_2['NDF_kg_DM']

0        6.417846
1        6.581358
2        7.344414
3        6.676740
4        8.311860
           ...   
15936    8.336520
15937    7.986924
15938    7.086042
15939    5.822118
15940    7.865910
Name: NDF_kg_DM, Length: 15941, dtype: float64

In [12]:
# # This function will retrieve the NDF (% DM) for the diet being fed and then multiply it by the 
# # cow's DMI to give the NDF intake in kg (DM), which is needed for the model

# def retrieve_ndf_value(row, conn):
#     cursor = conn.cursor()

#     diet_id = row['Diet_ID'] 

#     query = f"SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = '{diet_id}'"
#     cursor.execute(query)

#     column_data = cursor.fetchone()
#     # Divide by 100 to convert to a percentage
#     ndf_DM = float(column_data[0])/100

#     # Multiply by DMI to get the kg NDF consumed
#     ndf_value = ndf_DM * row['DMI']
#     return ndf_value

# conn = sqlite3.connect('../diet_database.db')

# # Apply the function to each row of the dataframe and assign the result to a new column
# animal_data['NDF_kg_DM'] = animal_data.apply(retrieve_ndf_value, args=(conn,), axis=1)

# conn.close()
